# LightFM

**Ответьте на вопросы:**

- В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?

*В коллаборативной фильтрации учитываются только взаимодействия user-item, и никак не учитываются их характеристики. Это делает сложным рекомендации для новых пользователей и товаров. В гибридных рекомендательных системах учитываются не только взаимодействия, но и признаки пользователей и товаров. Это позволяет улучшить качество рекомендации и делать рекомендации для "холодных" пользователей и товаров*


- Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы

*Гибридные системы лучше использовать там, где постоянно обновляется контент и появляется много новых пользователей. Например, маркетплейсы - Amazon, Ozon, Joom, там постоянно обновляются товалы, добавляются новые и исчезают старые, приходят новые пользователи. Или YouTube (новые видеозаписи), hh.ru (новые вакансии), также каждый день приходит много новых пользователей.*

Статья про поиск в hh.ru  - [habr](https://habr.com/ru/company/hh/blog/347276/)
WARP loss объяснение на игрушечном примере  - [статья](https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a)


In [1]:
#!pip install lightfm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [3]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_test.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2277416,338,41260573635,636,840173,1,1.99,369,0.0,112,92,0.0,0.0
2277417,338,41260573635,636,1037348,1,0.89,369,-0.3,112,92,0.0,0.0


In [4]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [5]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [6]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [7]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

## 1. Filter items

In [8]:
n_items_train_before = data_train['item_id'].nunique()
n_items_test_before = data_test['item_id'].nunique()
n_users_train_before = data_train['user_id'].nunique()
n_users_test_before = data_test['user_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)
data_test_filtered = prefilter_items(data_test, take_n_popular=5000, item_features=item_features)

n_items_train_after = data_train_filtered['item_id'].nunique()
n_items_test_after = data_test_filtered['item_id'].nunique()
n_users_train_after = data_train_filtered['user_id'].nunique()
n_users_test_after = data_test_filtered['user_id'].nunique()
print('Decreased # items_train from {} to {}'.format(n_items_train_before, n_items_train_after))
print('Decreased # items_test from {} to {}'.format(n_items_test_before, n_items_test_after))
print('Decreased # users_train from {} to {}'.format(n_users_train_before, n_users_train_after))
print('Decreased # users_test from {} to {}'.format(n_users_test_before, n_users_test_after))

Decreased # items_train from 86865 to 5001
Decreased # items_test from 24329 to 5001
Decreased # users_train from 2499 to 2497
Decreased # users_test from 2042 to 1990


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [9]:
user_item_matrix = pd.pivot_table(data_train_filtered,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count',
                                  fill_value=0
                                  )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [10]:
# Warm start - берем только те user_id, которые есть в train
data_test = data_test[data_test['user_id'].isin(data_train['user_id'])]

test_user_item_matrix = pd.pivot_table(data_test_filtered,
                                       index='user_id', columns='item_id',
                                       values='quantity', # Можно пробовать другие варианты
                                       aggfunc='count',
                                       fill_value=0
                                       )

test_user_item_matrix = test_user_item_matrix.astype(float)

test_sparse_user_item_matrix = csr_matrix(test_user_item_matrix).tocsr() # необходимый тип матрицы для implicit

test_user_item_matrix.head(2)

item_id,819210,819308,819423,819487,819840,819845,819978,820321,820347,820352,...,17381676,17381995,17827241,17900997,17901020,17903379,17903423,17959083,18000012,18024556
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [12]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(10)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown
8,25-34,U,25-34K,Unknown,2 Adults Kids,3,1
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# test_user_feat = pd.DataFrame(test_user_item_matrix.index)
# test_user_feat = test_user_feat.merge(user_features, on='user_id', how='left')
# test_user_feat.set_index('user_id', inplace=True)
# test_user_feat.head(10)

In [14]:
user_feat.shape\
    # , test_user_feat.shape

(2497, 7)

In [15]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(5)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ
819255,1007.0,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - PORK,1 LB
819308,2879.0,MEAT,National,BEEF,CHOICE BEEF,
819400,584.0,GROCERY,National,DOG FOODS,DOG TREATS (SOFT TREATS),5.6 OZ


In [16]:
# test_item_feat = pd.DataFrame(test_user_item_matrix.columns)
# test_item_feat = test_item_feat.merge(item_features, on='item_id', how='left')
# test_item_feat.set_index('item_id', inplace=True)
#
# test_item_feat.head(5)

In [17]:
item_feat.shape\
    # , test_item_feat.shape

(5001, 6)

## Encoding features

In [18]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [25]:
item_feat_lightfm.head(2)

,manufacturer_2.0,manufacturer_5.0,manufacturer_16.0,manufacturer_20.0,manufacturer_26.0,manufacturer_33.0,manufacturer_35.0,manufacturer_36.0,manufacturer_42.0,manufacturer_43.0,...,curr_size_of_product_L 16 OZ,curr_size_of_product_L 7.75 OZ,curr_size_of_product_L 13.25 OZ,curr_size_of_product_LB,curr_size_of_product_N 12 OZ,curr_size_of_product_N 40 OZ,curr_size_of_product_PINT,curr_size_of_product_PK,curr_size_of_product_PT,curr_size_of_product_QT
item_id,,,,,,,,,,,,,,,,,,,,,
117847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
818981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [24]:
model = LightFM(no_components=40,
                loss='bpr',
                # loss='warp',
                learning_rate=0.01,
                item_alpha=0.5,
                user_alpha=0.5,
                random_state=42)

## Train

In [25]:
%%time

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20,
          num_threads=4,
          verbose=False)

CPU times: user 1min 6s, sys: 576 ms, total: 1min 6s
Wall time: 1min 9s


## Evaluation

In [29]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [22]:
user_emb[0]  # biases

array([-11.819996,   0.      ,   0.      , ..., -12.279017,  -8.766911,
         0.      ], dtype=float32)

In [23]:
user_emb[1]  # embeddings

array([[ 6.29861496e-09, -3.14474846e-08,  8.39413705e-10, ...,
         1.00264250e-08, -7.14392856e-09, -2.89163680e-08],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 6.14127504e-09, -3.10411323e-08,  8.09362966e-10, ...,
         9.35850775e-09, -7.28431226e-09, -2.85422850e-08],
       [ 2.77240431e-09, -1.47063535e-08,  3.54995339e-10, ...,
         3.28663785e-09, -4.31271951e-09, -1.43793262e-08],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [24]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [25]:
item_emb[0].shape, item_emb[1].shape

((5001,), (5001, 40))

# Train precision

In [26]:
train_precision = precision_at_k(model, sparse_user_item,
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision = precision_at_k(model, test_sparse_user_item_matrix,
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision, test_precision

(0.21097319, 0.0024120605)

## Params Search

In [27]:
?LightFM

In [ ]:
# %%time
# no_components = [20, 30, 40, 50, 60]
# item_alpha = [1e-2, 1e-1, 0]
# user_alpha = [1e-2, 1e-1, 0]
# learning_rate = [1e-2, 5e-2 , 1e-1]
# loss = ['bpr', 'warp']
#
# result = pd.DataFrame(columns = ['no_components', 'item_alpha', 'user_alpha',
#                                  'learning_rate', 'loss', 'train_precision@5', 'test_precision@5'])
# i = 0
# for l in loss:
#     for c in no_components:
#         for lr in learning_rate:
#             for ia in item_alpha:
#                 for ua in user_alpha:
#                     try:
#                         model = LightFM(no_components=c,
#                                         loss=l,
#                                         learning_rate=lr,
#                                         item_alpha=ia,
#                                         user_alpha=ua,
#                                         random_state=42)
#
#                         model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
#                                   sample_weight=coo_matrix(user_item_matrix),
#                                   user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
#                                   item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
#                                   epochs=15,
#                                   num_threads=4,
#                                   verbose=False)
#
#                         train_precision = precision_at_k(model, sparse_user_item,
#                                                          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
#                                                          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
#                                                          k=5).mean()
#
#                         test_precision = precision_at_k(model, test_sparse_user_item_matrix,
#                                                          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
#                                                          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
#                                                          k=5).mean()
#
#                         result.loc[i] = [c, ia, ua, lr, l, train_precision, test_precision]
#
#                         print(f"{30*'-'}\nloss: {l}\nn_components: {c}\nitem_alpha: {ia}\nuser_alpha: {ua}\nlearning_rate: {lr}\ntrain precision@k: {train_precision:.4f}\ntest precision@k: {test_precision:.4f}")
#
#                     except ValueError:
#                         result.loc[i] = [c, ia, ua, lr, l, np.nan, np.nan]
#
#                         print(f'******Parameters failed:*****\n\loss: {l}\nn_components: {c}\nitem_alpha: {ia}\nuser_alpha: {ua}\nlearning_rate: {lr}')
#
#                     i+=1

******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0.01
user_alpha: 0.01
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0.01
user_alpha: 0.1
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0.01
user_alpha: 0
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.01
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.1
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0
user_alpha: 0.01
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0
user_alpha: 0.1
learning_rate: 0.01
******Parameters failed:*****
\loss: bpr
n_components: 20
item_alpha: 0
user_alpha: 0
learning_rate: 0.01
******Parameters

KeyboardInterrupt: 

In [62]:
result

,no_components,item_alpha,user_alpha,learning_rate,loss,train_precision@5,test_precision@5
0,20,0.01,0.01,0.01,bpr,0.419784,0.002613
1,20,0.01,0.10,0.01,bpr,0.199760,0.000603
2,20,0.01,0.00,0.01,bpr,0.347777,0.001910
3,20,0.10,0.01,0.01,bpr,0.201362,0.002010
4,20,0.10,0.10,0.01,bpr,0.438526,0.002814
...,...,...,...,...,...,...,...
265,60,0.10,0.10,0.10,warp,0.216099,0.001106
266,60,0.10,0.00,0.10,warp,0.276171,0.002513
267,60,0.00,0.01,0.10,warp,0.278574,0.003015
268,60,0.00,0.10,0.10,warp,0.061674,0.002613


In [28]:
%%time

# так как на тесте результат не очень хороший, попробуем еще поменять параметр epochs в fit
no_components = [20, 40, 60]
item_alpha = [1e-1, 5e-1, 8e-1]
user_alpha = [5e-2, 1e-1, 8e-1]
learning_rate = [1e-2, 5e-2 , 1e-1]
loss = ['bpr', 'warp']
epochs = [5, 10, 15, 20]

result = pd.DataFrame(columns = ['no_components', 'item_alpha', 'user_alpha',
                                 'learning_rate', 'loss', 'epochs', 'train_precision@5', 'test_precision@5'])
i = 0
for l in loss:
    for c in no_components:
        for lr in learning_rate:
            for ia in item_alpha:
                for ua in user_alpha:
                    model = LightFM(no_components=c,
                                    loss=l,
                                    learning_rate=lr,
                                    item_alpha=ia,
                                    user_alpha=ua,
                                    random_state=42)
                    for e in epochs:
                        try:
                            model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                                      sample_weight=coo_matrix(user_item_matrix),
                                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                      epochs=e,
                                      num_threads=4,
                                      verbose=False)

                            train_precision = precision_at_k(model, sparse_user_item,
                                                             user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                                             item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                                             k=5).mean()

                            test_precision = precision_at_k(model, test_sparse_user_item_matrix,
                                                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                                            k=5).mean()

                            result.loc[i] = [c, ia, ua, lr, l, e, train_precision, test_precision]

                            print(f"{30*'-'}\nloss: {l}\nn_components: {c}\nitem_alpha: {ia}\nuser_alpha: {ua}\nlearning_rate: {lr}\nepochs: {e}\ntrain precision@k: {train_precision:.4f}\ntest precision@k: {test_precision:.4f}")

                        except ValueError:
                            result.loc[i] = [c, ia, ua, lr, l, e, np.nan, np.nan]

                            print(f'******Parameters failed:*****\n\loss: {l}\nn_components: {c}\nitem_alpha: {ia}\nuser_alpha: {ua}\nlearning_rate: {lr}\nepoch: {e}')

                        i+=1

------------------------------
loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.05
learning_rate: 0.01
epochs: 5
train precision@k: 0.4385
test precision@k: 0.0028
------------------------------
loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.05
learning_rate: 0.01
epochs: 10
train precision@k: 0.4385
test precision@k: 0.0028
------------------------------
loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.05
learning_rate: 0.01
epochs: 15
train precision@k: 0.4385
test precision@k: 0.0028
------------------------------
loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.05
learning_rate: 0.01
epochs: 20
train precision@k: 0.4385
test precision@k: 0.0028
------------------------------
loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.1
learning_rate: 0.01
epochs: 5
train precision@k: 0.4385
test precision@k: 0.0028
------------------------------
loss: bpr
n_components: 20
item_alpha: 0.1
user_alpha: 0.1
learning_rate: 0.01
epochs: 10
train precision@k

In [29]:
result

,no_components,item_alpha,user_alpha,learning_rate,loss,epochs,train_precision@5,test_precision@5
0,20,0.1,0.05,0.01,bpr,5,0.438526,0.002814
1,20,0.1,0.05,0.01,bpr,10,0.438526,0.002814
2,20,0.1,0.05,0.01,bpr,15,0.438526,0.002814
3,20,0.1,0.05,0.01,bpr,20,0.438526,0.002814
4,20,0.1,0.10,0.01,bpr,5,0.438526,0.002814
...,...,...,...,...,...,...,...,...
643,60,0.8,0.10,0.10,warp,20,0.199920,0.000905
644,60,0.8,0.80,0.10,warp,5,0.208410,0.002915
645,60,0.8,0.80,0.10,warp,10,0.131438,0.001508
646,60,0.8,0.80,0.10,warp,15,0.147537,0.002010


In [34]:
result.loc[result['test_precision@5'].values > 0.015]

,no_components,item_alpha,user_alpha,learning_rate,loss,epochs,train_precision@5,test_precision@5
48,20,0.5,0.05,0.05,bpr,5,0.205046,0.015477
52,20,0.5,0.10,0.05,bpr,5,0.205046,0.015477
70,20,0.8,0.80,0.05,bpr,15,0.226432,0.018894
245,60,0.8,0.10,0.01,bpr,10,0.202563,0.015779
